In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

load_dotenv(dotenv_path=Path("..") / ".env")

print("LLM_PROVIDER:", os.environ.get("LLM_PROVIDER"))
print("GEMINI_MODEL:", os.environ.get("GEMINI_MODEL"))
print("Has GEMINI key:", bool(os.environ.get("GEMINI_API_KEY")))


LLM_PROVIDER: gemini
GEMINI_MODEL: gemini-2.5-flash
Has GEMINI key: True


In [2]:
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
sys.path.append(str(PROJECT_ROOT))

from services.parser import parse_reviews_file
from services.llm_client import analyze_reviews_with_llm
from services.report_pdf import build_pdf


In [3]:
from pathlib import Path

xlsx_path = Path("..") / "data" / "reviews_test.xlsx"
print("Exists:", xlsx_path.exists(), "Path:", xlsx_path.resolve())

file_bytes = xlsx_path.read_bytes()

reviews = parse_reviews_file(filename="reviews_test.xlsx", content=file_bytes)

print("Parsed reviews:", len(reviews))
print("First item keys:", reviews[0].keys() if reviews else None)
print("First review sample:", reviews[0] if reviews else None)


Exists: True Path: /home/nikita/code/PlatoIsDead/hotel_review_analyzer/data/reviews_test.xlsx
Parsed reviews: 102
First item keys: dict_keys(['review_text'])
First review sample: {'review_text': 'New Horizon;74387443927;"2025-12-03 11:00:02";"Мило'}


In [4]:
# show a few review texts
for i in range(min(5, len(reviews))):
    print(i, reviews[i].get("review_text", "")[:200], "\n---\n")


0 New Horizon;74387443927;"2025-12-03 11:00:02";"Мило 
---

1 Наквартире;710135160720;"2025-12-03 09:00:02";"На фото недостатки выявленные при заселении и проживании. В остальном все хорошо";4;"Оцените качество размещения" 
---

2 Анна/AS;786134818546;"2025-12-03 09:00:04";"Чистая уютная студия все в пешей доступности.";5;"Оцените качество размещения" 
---

3 RentWill;8026135545948;"2025-12-03 12:00:01";"Чут холодно а так всё супер спасибо ";5;"Дорогой гость! 💚 
---

4 Оцените 
---



In [5]:
report = analyze_reviews_with_llm(reviews=reviews[:10], custom_prompt="")
report.keys() if isinstance(report, dict) else type(report)


dict_keys(['executive_summary', 'key_themes', 'positives', 'negatives', 'actionable_recommendations', 'risk_flags'])

In [6]:
import os, requests

key = os.environ["GEMINI_API_KEY"]
url = f"https://generativelanguage.googleapis.com/v1beta/models?key={key}"

r = requests.get(url, timeout=30)
print("status:", r.status_code)
print(r.text[:1000])


status: 200
{
  "models": [
    {
      "name": "models/embedding-gecko-001",
      "version": "001",
      "displayName": "Embedding Gecko",
      "description": "Obtain a distributed representation of a text.",
      "inputTokenLimit": 1024,
      "outputTokenLimit": 1,
      "supportedGenerationMethods": [
        "embedText",
        "countTextTokens"
      ]
    },
    {
      "name": "models/gemini-2.5-flash",
      "version": "001",
      "displayName": "Gemini 2.5 Flash",
      "description": "Stable version of Gemini 2.5 Flash, our mid-size multimodal model that supports up to 1 million tokens, released in June of 2025.",
      "inputTokenLimit": 1048576,
      "outputTokenLimit": 65536,
      "supportedGenerationMethods": [
        "generateContent",
        "countTokens",
        "createCachedContent",
        "batchGenerateContent"
      ],
      "temperature": 1,
      "topP": 0.95,
      "topK": 64,
      "maxTemperature": 2,
      "thinking": true
    },
    {
      "nam